# Robust Optimization: Learning Uncertainty Sets

This notebook demonstrates **robust optimization** with CVXPYlayers — designing solutions that perform well under worst-case uncertainty, and *learning* the right amount of robustness from data.

**Problem**: Robust least-squares regression. Given uncertain data matrix $A$, we solve:

$$\min_x \; \|Ax - b\|_2^2 + \gamma \cdot \epsilon \cdot \|x\|_2$$

The regularization term $\gamma \cdot \epsilon \cdot \|x\|_2$ is the robust counterpart for spectral-norm bounded perturbations $\|\Delta\| \leq \epsilon$. The uncertainty radius $\epsilon$ is a learnable parameter trained to minimize out-of-sample prediction error.

**Key idea**: Too little robustness ($\epsilon \approx 0$) overfits; too much robustness underperforms. CVXPYlayers lets us *learn* the right $\epsilon$ by backpropagating through the optimization.

In [ ]:
import cvxpy as cp
import matplotlib.pyplot as plt
import numpy as np
import torch

from cvxpylayers.torch import CvxpyLayer

## Generate Synthetic Data with Perturbations

We create training data from a linear model $y = A_{\text{true}} x_{\text{true}} + \text{noise}$, but the observed $A$ has measurement errors.

In [ ]:
np.random.seed(42)

n_features = 10
n_train = 30
n_val = 200
noise_std = 0.3
perturbation_std = 0.15  # measurement error in A

# True model
x_true = np.random.randn(n_features)
x_true /= np.linalg.norm(x_true)

# True (clean) data matrix
A_true = np.random.randn(n_train, n_features)
b_train = A_true @ x_true + noise_std * np.random.randn(n_train)

# Observed (perturbed) training data
A_train = A_true + perturbation_std * np.random.randn(n_train, n_features)

# Validation data (clean A, noisy b)
A_val = np.random.randn(n_val, n_features)
b_val = A_val @ x_true + noise_std * np.random.randn(n_val)

print(f"Training: {n_train} samples, {n_features} features")
print(f"Validation: {n_val} samples")
print(f"Perturbation std: {perturbation_std}")

## Define Robust Least-Squares Layer

The robust counterpart of least-squares with spectral norm uncertainty $\|\Delta\| \leq \epsilon$ adds a regularization term $\epsilon \|x\|_2$. This is equivalent to Tikhonov regularization, but with a principled interpretation as worst-case robustness.

We parametrize the problem with:
- `b_param` — the observed response vector
- `epsilon` — the uncertainty radius (to be learned)

In [ ]:
x_var = cp.Variable(n_features, name="x")
b_param = cp.Parameter(n_train, name="b")
epsilon = cp.Parameter(nonneg=True, name="epsilon")

# A_train is fixed (not a parameter for simplicity)
A_np = A_train.copy()

# Robust counterpart: ||Ax - b||^2 + epsilon * ||x||_2
objective = cp.Minimize(
    cp.sum_squares(A_np @ x_var - b_param) + epsilon * cp.norm(x_var, 2)
)
problem = cp.Problem(objective)
assert problem.is_dpp()

layer = CvxpyLayer(problem, parameters=[b_param, epsilon], variables=[x_var])
print("Robust least-squares layer created.")

## Solve for Different Uncertainty Radii

Sweep $\epsilon$ to see how robustness affects training and validation error.

In [ ]:
b_tch = torch.tensor(b_train, dtype=torch.float64)
A_val_tch = torch.tensor(A_val, dtype=torch.float64)
b_val_tch = torch.tensor(b_val, dtype=torch.float64)
A_train_tch = torch.tensor(A_train, dtype=torch.float64)

eps_values = np.logspace(-2, 2, 30)
train_errors = []
val_errors = []

for eps_val in eps_values:
    eps_tch = torch.tensor(eps_val, dtype=torch.float64)
    (x_sol,) = layer(b_tch, eps_tch)
    
    train_err = torch.mean((A_train_tch @ x_sol - b_tch) ** 2).item()
    val_err = torch.mean((A_val_tch @ x_sol - b_val_tch) ** 2).item()
    train_errors.append(train_err)
    val_errors.append(val_err)

best_idx = np.argmin(val_errors)
print(f"Best epsilon (grid search): {eps_values[best_idx]:.4f}")
print(f"Best validation MSE:        {val_errors[best_idx]:.4f}")

## Learn Epsilon via SGD

Instead of grid search, use gradient descent to find the optimal $\epsilon$. CVXPYlayers differentiates through the solve, so we can directly minimize validation error.

In [ ]:
# Learnable epsilon (log-parameterized for positivity)
log_eps = torch.tensor(0.0, dtype=torch.float64, requires_grad=True)
optimizer = torch.optim.Adam([log_eps], lr=0.1)

losses = []
eps_trajectory = []

for step in range(50):
    optimizer.zero_grad()
    eps_tch = torch.exp(log_eps)
    (x_sol,) = layer(b_tch, eps_tch)
    
    # Minimize validation MSE
    val_loss = torch.mean((A_val_tch @ x_sol - b_val_tch) ** 2)
    val_loss.backward()
    optimizer.step()
    
    losses.append(val_loss.item())
    eps_trajectory.append(eps_tch.item())

learned_eps = torch.exp(log_eps).item()
print(f"Learned epsilon: {learned_eps:.4f}")
print(f"Final validation MSE: {losses[-1]:.4f}")

## Results: Robust vs. Nominal

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Plot 1: Train/val error vs epsilon
ax = axes[0]
ax.semilogx(eps_values, train_errors, 'b-', label='Train MSE', linewidth=2)
ax.semilogx(eps_values, val_errors, 'r-', label='Validation MSE', linewidth=2)
ax.axvline(x=learned_eps, color='g', linestyle='--', label=f'Learned ε={learned_eps:.2f}')
ax.axvline(x=eps_values[best_idx], color='orange', linestyle=':', label=f'Grid best ε={eps_values[best_idx]:.2f}')
ax.set_xlabel('Uncertainty Radius (ε)')
ax.set_ylabel('MSE')
ax.set_title('Error vs. Robustness')
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# Plot 2: SGD convergence
ax = axes[1]
ax.plot(losses, 'g-', linewidth=2)
ax.set_xlabel('SGD Step')
ax.set_ylabel('Validation MSE')
ax.set_title('Learning Curve')
ax.grid(True, alpha=0.3)

# Plot 3: Epsilon trajectory
ax = axes[2]
ax.plot(eps_trajectory, 'g-', linewidth=2)
ax.axhline(y=eps_values[best_idx], color='orange', linestyle=':', label='Grid search best')
ax.set_xlabel('SGD Step')
ax.set_ylabel('ε')
ax.set_title('Learned ε Trajectory')
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Compare nominal vs robust
eps_nominal = torch.tensor(0.0, dtype=torch.float64)
(x_nominal,) = layer(b_tch, eps_nominal)
eps_robust = torch.tensor(learned_eps, dtype=torch.float64)
(x_robust,) = layer(b_tch, eps_robust)

nom_val_err = torch.mean((A_val_tch @ x_nominal - b_val_tch) ** 2).item()
rob_val_err = torch.mean((A_val_tch @ x_robust - b_val_tch) ** 2).item()

print(f"Nominal (ε=0) validation MSE:   {nom_val_err:.4f}")
print(f"Robust (ε={learned_eps:.2f}) validation MSE: {rob_val_err:.4f}")
print(f"Improvement: {(nom_val_err - rob_val_err) / nom_val_err * 100:.1f}%")